<a href="https://colab.research.google.com/github/mlKwon/Multi_Modal_Fusion_ERC/blob/main/try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp39-cp39-linux_x86_64.whl size=680526 sha256=0c173c7d909ceaa48b78f34d90423e22b7d45049accf4251ea3e9bb19622959d
  Stored in directory: /root/.cache/pip/wheels/47/17/70/b257bc53879a458c4bfcc900e89271aa8b4f19366a54bd2455
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [3]:
import os
import glob
import gc

os.chdir("drive/MyDrive/2023aifactory/wav")
dirpath = os.path.join(os.getcwd(),os.listdir()[0])
audio_path = glob.glob(dirpath + "/*.txt")

audio_path

['/content/drive/MyDrive/2023aifactory/wav/Session01/Sess01_script06_User002M_028.txt',
 '/content/drive/MyDrive/2023aifactory/wav/Session01/Sess01_script06_User002M_038.txt',
 '/content/drive/MyDrive/2023aifactory/wav/Session01/Sess01_script06_User002M_043.txt',
 '/content/drive/MyDrive/2023aifactory/wav/Session01/Sess01_script06_User002M_033.txt',
 '/content/drive/MyDrive/2023aifactory/wav/Session01/Sess01_script06_User002M_029.txt',
 '/content/drive/MyDrive/2023aifactory/wav/Session01/Sess01_script06_User002M_026.txt',
 '/content/drive/MyDrive/2023aifactory/wav/Session01/Sess01_script06_User002M_030.txt',
 '/content/drive/MyDrive/2023aifactory/wav/Session01/Sess01_script06_User002M_040.txt',
 '/content/drive/MyDrive/2023aifactory/wav/Session01/Sess01_script06_User002M_034.txt',
 '/content/drive/MyDrive/2023aifactory/wav/Session01/Sess01_script06_User002M_037.txt',
 '/content/drive/MyDrive/2023aifactory/wav/Session01/Sess01_script06_User002M_031.txt',
 '/content/drive/MyDrive/2023aif

In [4]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-0z6qkfps/kobert-tokenizer_1b9a023a894e46ceb671ebb654faeb22
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-0z6qkfps/kobert-tokenizer_1b9a023a894e46ceb671ebb654faeb22
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4645 sha256=3f87fcbf89492acda536e2326a23513cb66a870ceebb64b898341c7d6d8288c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-4vc_uw71/wheels/64/c4/82/d70d864682468fad0bf8269b0dfd88daf48f209be4c8ade164
Successfully built kobert_tokenizer


In [5]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW # 인공지능 모델의 초기값 지정 함수를 아담으로 지정한다.
from transformers.optimization import get_cosine_schedule_with_warmup

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

# model, vocab = get_pytorch_kobert_model()

from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
torch.cuda.device_count()
if torch.cuda.is_available():    
    device = torch.device("cuda")

# tokenizer = get_tokenizer()
vocab[0]

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

0

In [6]:
import pandas as pd
import re
annot = pd.read_csv('/content/drive/MyDrive/2023aifactory/annotation/Sess01_eval.csv')
id = annot.iloc[1:,3].tolist()
speaker = ["Female" for i in range(len(id))]
for i in range(len(id)):
  if str.find(id[i],"M") > 0:
    speaker[i] = "Male"
emotion = annot.iloc[1:,4]
emotion[emotion.str.contains(";")] = emotion[emotion.str.contains(";")].str.split(";").str[0] # Choose first emotion when there are two emotions.
emotion = emotion.tolist()
valence = annot.iloc[1:,5].tolist()
arousal = annot.iloc[1:,6].tolist()


In [7]:
dic_ = {'happy':0,'surprise':1,'angry':2,'neutral':3,'disqust':4,'fear':5,'sad':6}

def removeTag(text):
  text = text.replace("c/","")
  text = text.replace("n/","")
  text = text.replace("N/","")
  text = text.replace("u/","")
  text = text.replace("l/","")
  text = text.replace("b/","")
  text = text.replace("*","")
  text = text.replace("+","")
  text = text.replace("/","")
  text = text.replace("\n","")
  text = text.replace("/","  ").replace("/","  ").replace("/","  ").lstrip().rstrip()
  return text

In [8]:
lst = [['',0] for i in range(len(id)) ]

for i in range(len(id)):  
  # dat=nlp.data.TSVDataset(audio_path[0], encoding='cp949')
  f = open(os.getcwd() + "/Session01/"+id[i]+".txt",'r', encoding='cp949')
  line = f.readline()
  lst[i][0] = removeTag(line)
  lst[i][1] = dic_[emotion[i]]
  # print(lst)
  f.close()

In [9]:
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')

for i in range(len(id)):
  text_ = [lst[i][0]]
  inputs = tokenizer.batch_encode_plus(text_)
  out = model(input_ids = torch.tensor(inputs['input_ids']),
              attention_mask = torch.tensor(inputs['attention_mask']))
  if i==0: kobert_output = out.pooler_output
  else: kobert_output = torch.concat([kobert_output, out.pooler_output],axis=0)

kobert_output.shape

Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

torch.Size([311, 768])

In [ ]:
model.num_parameters
max(kobert_output[0,:])

tensor(0.9986, grad_fn=<UnbindBackward0>)

In [10]:
## temporally reduce dimension of text input
def removeDim(input,output_dim=64):
  input_dim=input.shape[1]
  if input.count_nonzero() > 0:
    func = nn.Linear(input_dim,output_dim)
  else: ## audio vector with no sound ## fix all value to zero
    func = nn.Linear(input_dim,output_dim, bias=False)

  return func(input).squeeze()

In [11]:
mod_kobert_output = removeDim(kobert_output)

In [ ]:
mod_kobert_output.shape

torch.Size([311, 64])

MFCC

In [12]:
# !pip install librosa
import librosa
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import itertools

def get_librosa_mfcc(filepath, n_mfcc=64):
    sample_rate = 16000
    hop_length = 160
    n_fft = 400
    
    sig, sr = librosa.core.load(filepath, sr=sample_rate)
    mfccs = librosa.feature.mfcc(y=sig, sr=sr, hop_length=hop_length,
                                 n_mfcc=n_mfcc, n_fft=n_fft)
    mfccs = torch.FloatTensor(mfccs).transpose(0,1)
    return mfccs

from sklearn.decomposition import PCA
pca = PCA(n_components=1)

In [13]:
pca_ratio = [0 for i in range(len(id))]
mfcc_output = torch.zeros([len(id),64])
mfcc_output2 = torch.zeros([len(id),64])
print(mfcc_output.shape)

for i in range(len(id)):
    filepath =  os.getcwd() + "/Session01/"+id[i]+".wav"
    temp = get_librosa_mfcc(filepath)
    temp = preprocessing.scale(temp, axis=0)
    mfcc_output2[i,:] = removeDim(torch.tensor(temp.T,dtype=torch.float32),output_dim=1).T
    printcipalComponents = pca.fit_transform(temp.T)
    pca_ratio[i] = pca.explained_variance_ratio_
    mfcc_output[i,:] = torch.tensor(list(itertools.chain(*printcipalComponents)))
    # librosa.display.specshow(temp,sr=16000,x_axis='time')
    # plt.show()
pca_ratio = list(itertools.chain(*pca_ratio))

torch.Size([311, 64])


/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_pca.py:642: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.9/

In [ ]:
print(mfcc_output2[215,:])
mfcc_output[215,:]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       grad_fn=<SliceBackward0>)


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
# session1 audio error id: [215, 216, 219, 222, 223, 224, 225, 285, 293, 295]
ddd = []
# np.where(np.isnan(pca_ratio))
for i in [215, 216, 219, 222, 223, 224, 225, 285, 293, 295]:
    filepath =  os.getcwd() + "/Session01/"+id[i]+".wav"
    temp = get_librosa_mfcc(filepath)
    temp = preprocessing.scale(temp, axis=0)
    print(temp.shape)
    ddd.append( pca.fit_transform(temp.T) )



After extract text and audio features, do the attention

In [14]:
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn.functional as F
import math
from torch.nn import GRUCell

# class GRUModel(nn.Module) :
#     def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True) :
#         super(GRUModel, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.layer_dim = layer_dim
#         self.gru_cell = nn.GRUCell(input_dim, hidden_dim, layer_dim)
#         self.fc = nn.Linear(hidden_dim, output_dim)
        
#     def forward(self, x) :
#         if torch.cuda.is_available() :
#             h0 = torch.tensor(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
#         else :
#             h0 = torch.tensor(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
#         outs = []
#         hn = h0[0, :, :]
        
#         for seq in range(x.size(1)) : 
#             hn = self.gru_cell(x[:, seq, :], hn)
#             outs.append(hn)
#         out = outs[-1].squeeze()
#         out = self.fc(out)
#         return out

Do GRU

In [ ]:
torch.randn(2, 3, 20)[:,1,:]

tensor([[ 1.1319, -0.0656,  0.1286, -0.3348, -1.7161, -0.3717, -1.2334,  0.0637,
          0.2079,  1.5338,  0.2719,  0.3298, -0.1631,  0.6489, -0.2520, -0.8150,
         -1.2041, -0.2126, -0.1426, -2.2703],
        [-0.5612,  0.9102,  0.6920, -0.3841,  0.8797, -1.4576, -0.6450, -0.4510,
          0.3431, -1.0804, -0.3721,  1.2028,  0.3713,  0.2103,  0.0427, -0.7856,
          0.6621,  0.2276,  0.4172,  0.0402]])

In [15]:
from torch.nn import GRU, GRUCell
import copy
from copy import deepcopy
# class 

In [16]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, batch_dim=1, dropout_p=0.2, bContext=False, gender=0):
        super(GRUModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p) 
        self.layer_dim = layer_dim
        self.batch_dim = batch_dim
        self.rnn = nn.GRU(input_dim, hidden_dim, layer_dim)
        self.bContext = bContext # bContext: Context level
        self.gender = gender

        # 3 layers of gru
        self.gru_0 = nn.GRUCell(input_dim, hidden_dim)
        self.gru_1 = nn.GRUCell(hidden_dim, hidden_dim)
        self.gru_2 = nn.GRUCell(hidden_dim, hidden_dim)
        
    def forward(self, x):
        print(self.bContext)
        # Initialize hidden state with zeros
        #######################
        #  USE GPU FOR MODEL  #
        #######################

        if torch.cuda.is_available():
          x = x.cuda()
          # h0 = torch.randn(self.layer_dim, self.batch_dim, self.hidden_dim).cuda() # layer, batch, hid # for GRU
          h0 = torch.randn(self.batch_dim, self.hidden_dim).cuda() # layer, hid
          h1 = torch.randn(self.batch_dim, self.hidden_dim).cuda() # layer, hid
          h2 = torch.randn(self.batch_dim, self.hidden_dim).cuda() # layer, hid
          h0_F = torch.randn(self.batch_dim, self.hidden_dim).cuda() # for self-context level # Female
          h1_F = torch.randn(self.batch_dim, self.hidden_dim).cuda() # for self-context level # Female
          h2_F = torch.randn(self.batch_dim, self.hidden_dim).cuda() # for self-context level # Female
        else:
          # h0 = torch.randn(self.layer_dim, 1, self.hidden_dim)
          h0 = torch.randn(self.batch_dim, self.hidden_dim)
          h1 = torch.randn(self.batch_dim, self.hidden_dim)
          h2 = torch.randn(self.batch_dim, self.hidden_dim)
          h0_F = torch.randn(self.batch_dim, self.hidden_dim) # for self-context level # Female
          h1_F = torch.randn(self.batch_dim, self.hidden_dim) # for self-context level # Female
          h2_F = torch.randn(self.batch_dim, self.hidden_dim) # for self-context level # Female
       
        outs = []

        for i in range(x.shape[0]): # 1~seq_len
            input_ = x[i,:,:]
            if self.bContext==True: # context-level
              if input_.count_nonzero() > 0: # 정상 utterance vector
                h0 = self.gru_0(input_,h0)
                h1 = self.gru_1(h1,h0)
                h2 = self.gru_2(h2,h1)
                outs.append(h2.unsqueeze(0))
              else: # utterance vector 값 없을 때 # audio in session1
                # print(i)
                outs.append(torch.zeros_like(h1.unsqueeze(0)))
            else: # self-context level
              if input_.count_nonzero() > 0: # 정상 utterance vector
                if self.gender[i]==0: # speaker gender
                  h0 = self.gru_0(input_,h0)
                  h1 = self.gru_1(h0,h1)
                  h2 = self.gru_2(h1,h2)
                else: 
                  h0_F = self.gru_0(input_,h0_F)
                  h1_F = self.gru_1(h0_F,h1_F)
                  h2_F = self.gru_2(h1_F,h2_F)
                outs.append(h2_F.unsqueeze(0))
              else: # utterance vector 값 없을 때 # audio in session1
                # print(i)
                outs.append(torch.zeros_like(h1.unsqueeze(0)))
            
            
        
        # hn = deepcopy(h0)
        # outs, hn = self.rnn(x, h0)

        # outs = outs.squeeze()
        outs = torch.cat(outs, dim = 0).squeeze()
        
        return outs

In [17]:
# text model
model = GRUModel(input_dim = 64, hidden_dim=32, layer_dim=2, output_dim=32, dropout_p=0.2, bContext=True)
le = preprocessing.LabelEncoder()
gender=torch.tensor(le.fit_transform(speaker))
model_s = GRUModel(input_dim = 64, hidden_dim=32, layer_dim=2, output_dim=32, dropout_p=0.2, bContext=False, gender=gender)
if torch.cuda.is_available():
  model = model.cuda()
  model_s = model_s.cuda()
kobert_rnn_c = model(mod_kobert_output.unsqueeze(1))
mfcc_rnn_c = model(mfcc_output2.unsqueeze(1))

kobert_rnn_s = model_s(mod_kobert_output.unsqueeze(1))
mfcc_rnn_s = model_s(mfcc_output2.unsqueeze(1))

True
True
False
False


In [ ]:
import gc
gc.collect()

531

In [18]:
# kobert_rnn.shape
print(kobert_rnn_c[215,:])
print(mfcc_rnn_c[215,:])

print(kobert_rnn_s[215,:])
print(mfcc_rnn_s[215,:])
print(kobert_rnn_s.shape)
print(mfcc_rnn_s.shape)

kobert_rnn = (kobert_rnn_s+kobert_rnn_c)/2
mfcc_rnn = (mfcc_rnn_s+mfcc_rnn_c)/2

del kobert_rnn_c,kobert_rnn_s,mfcc_rnn_c,mfcc_rnn_s
gc.collect()

tensor([-0.0247, -0.1147,  0.0609, -0.0067, -0.0950,  0.0182,  0.0104, -0.0019,
        -0.0142, -0.0084,  0.1748,  0.0129,  0.0483,  0.0024, -0.0439, -0.0097,
         0.0477,  0.0961,  0.0746,  0.0002,  0.1076, -0.0029, -0.1910, -0.0339,
         0.1622,  0.0230, -0.0714, -0.0184,  0.0127, -0.0372,  0.0011, -0.0960],
       device='cuda:0', grad_fn=<SliceBackward0>)
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([-0.1537,  0.0526,  0.1103,  0.2760,  0.2627,  0.0973,  0.0065,  0.0161,
         0.2275, -0.1567,  0.1238,  0.1544,  0.1516,  0.0349,  0.2485,  0.1229,
         0.0162,  0.0715,  0.0155, -0.0072, -0.4768, -0.2629, -0.3428,  0.2098,
         0.0490,  0.2053,  0.2870, -0.2895, -0.2346,  0.1198, -0.1264,  0.0729],
       device='cuda:0', grad_fn=<SliceBackward0>)
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

4228

to do list

0. feature extraction from audio and text -> OK
1. c-lstm and s-lstm -> OK
2. bi-gru
3. transformer (only encoding)
4. define loss function
5. make whole model (define batch, class folder)
6. Use biosensor data (how to match)